In [2]:
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from typing import *
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题
my_seed = 1337
random.seed(my_seed)
np.random.seed(my_seed)

### 简单数据查看

In [3]:

def load_movies_dataset():
    #movie_data_columns = [
    #'type_douban', 'actor', 'region', 'director', 'characteristic',
    #'score_douban', 'moviename']
    movie_data = pd.read_csv(data_dir + './dataset1/movie.csv')
    #movie_data.columns = movie_data_columns 
    return movie_data 

def load_user_and_ratings() :
    user_data = pd.read_csv(data_dir + './dataset1/user.csv')
    user_data['comment_time'] = pd.to_datetime(user_data['评论时间'])
    return user_data 

def load_doubanmovieshortcomments() :
    movieshortcomments_data = pd.read_csv(data_dir_doubanmovieshortcomments + './DMSC.csv')

    return movieshortcomments_data 

data_dir = 'D:/python/Jupyter_Last_project/dataset/'
data_dir_doubanmovieshortcomments = 'D:/python/Jupyter_Last_project/dataset/doubanmovieshortcomments/'
movies_data = load_movies_dataset()
print('movies_data.head()\n',movies_data.head())
#movieshortcomments_data = load_doubanmovieshortcomments()
#print('movieshortcomments_data.head()\n',movieshortcomments_data.head())
user_data = load_user_and_ratings() 

del user_data['评论时间']
print('user_data.head()\n',user_data.head())

movies_data.head()
    类型                  主演    地区     导演  特色   评分     电影名
0  剧情    徐峥|王传君|周一围|谭卓|章宇  中国大陆    文牧野  经典  8.9   我不是药神
1  剧情   冯小刚|许晴|张涵予|刘桦|李易峰  中国大陆     管虎  经典  7.8     老炮儿
2  剧情  王宝强|刘昊然|肖央|刘承羽|尚语贤  中国大陆    陈思诚  经典  6.7  唐人街探案2
3  剧情  任素汐|大力|刘帅良|裴魁山|阿如那  中国大陆  周申|刘露  经典  8.3     驴得水
4  剧情   徐峥|王宝强|李曼|李小璐|左小青  中国大陆    叶伟民  经典  7.5    人在囧途
user_data.head()
    评分     用户名  用户ID    电影名  类型        comment_time
0   2      身似     1   心雨花露  爱情 2018-01-05 15:05:06
1   4  有意识的贱民     3  战争的恐怖  战争 2018-01-05 15:05:06
2   2    亿万露电     4  豪勇七蛟龙  战争 2018-01-05 15:05:06
3   2   Marni     5   无序之主  犯罪 2018-01-05 15:05:06
4   4   马西嘻嘻嘻     6  时装店风波  同性 2018-01-05 15:05:06


In [4]:
user_data.shape

(188843, 6)

In [5]:
user_data['用户ID'].nunique()

13545

In [6]:
user_data['电影名'].nunique()

21753

In [7]:
user_data['评分'].value_counts()

8     75078
4     50685
10    49548
2     13520
6        12
Name: 评分, dtype: int64

### 按用户id分组。得到每个用户评分的均值，标准差，最大值，最小值，中位数

In [8]:
user_data.head(2)

,评分,用户名,用户ID,电影名,类型,comment_time
0,2,身似,1,心雨花露,爱情,2018-01-05 15:05:06
1,4,有意识的贱民,3,战争的恐怖,战争,2018-01-05 15:05:06


In [9]:
df02 = user_data.groupby(['用户ID']).评分.agg( {'user_rating_avg':np.mean ,
                                           'user_rating_max':np.max ,
                                           'user_rating_min':np.min ,
                                           'user_rating_median':np.median }).reset_index()
print(df02.head(2))

   用户ID  user_rating_avg  user_rating_max  user_rating_min  user_rating_median
0     1         8.400000               10                2                  10
1     3         6.984848               10                2                   8


In [10]:
print(user_data[user_data['类型'] == '传记'].sort_values(by='用户ID').head(5))

        评分     用户名  用户ID      电影名  类型        comment_time
70373    2  有意识的贱民     3  加尔各答的天使  传记 2018-01-19 21:03:22
58693    4  有意识的贱民     3    城市之光2  传记 2018-01-19 16:11:22
161596   4    亿万露电     4     黑蝶漫舞  传记 2018-01-28 15:02:20
98735   10   Marni     5    无处的男孩  传记 2018-01-23 12:16:17
20190    8   马西嘻嘻嘻     6     旅行终点  传记 2018-01-08 12:55:07


### ---------------按用户id和电影名分组

### 按类型，用户id分组。得到同一类型下相同用户评分的均值，最大值，最小值，中位数

In [11]:
df03 = user_data.groupby(['类型','用户ID']).评分.agg( {'type_user_rating_avg':np.mean ,
                                           'type_user_rating_max':np.max ,
                                           'type_user_rating_min':np.min ,
                                           'type_user_rating_median':np.median }).reset_index()
print(df03.head( ))
print(df03.shape)

   类型  用户ID  type_user_rating_avg  type_user_rating_max  type_user_rating_min  \
0  传记     3              3.000000                     4                     2   
1  传记     4              4.000000                     4                     4   
2  传记     5             10.000000                    10                    10   
3  传记     6              8.000000                     8                     8   
4  传记     8              7.333333                    10                     4   

   type_user_rating_median  
0                        3  
1                        4  
2                       10  
3                        8  
4                        8  
(96313, 6)


In [12]:
user_data.shape

(188843, 6)

In [13]:
movies_data.shape

(89524, 7)

In [14]:
movies_data['电影名'].nunique()

22971

In [15]:
movies_data['导演'].nunique()

13831

In [16]:
movies_data['特色'].nunique()

9

In [17]:
movies_data['地区'].nunique()

21

In [18]:
movies_data['类型'].nunique()

19

In [19]:
movies_data.rename(columns = {"评分": "豆瓣网评分"},inplace=True)
print(movies_data.head(2))


   类型                 主演    地区   导演  特色  豆瓣网评分    电影名
0  剧情   徐峥|王传君|周一围|谭卓|章宇  中国大陆  文牧野  经典    8.9  我不是药神
1  剧情  冯小刚|许晴|张涵予|刘桦|李易峰  中国大陆   管虎  经典    7.8    老炮儿


In [20]:
df3 = movies_data.groupby(['电影名']).豆瓣网评分.agg( {'movie_name_rating_avg':np.mean ,
                                           'movie_name_max':np.max ,
                                           'movie_name_min':np.min ,
                                           'movie_name_median':np.median }).reset_index()
print(df3.head())

         电影名  movie_name_rating_avg  movie_name_max  movie_name_min  \
0        0.5                    5.6             8.2             4.3   
1   007之你死我活                    6.7             6.7             6.7   
2  007之俄罗斯之恋                    7.2             7.2             7.2   
3    007之八爪女                    6.8             6.8             6.8   
4    007之太空城                    6.9             6.9             6.9   

   movie_name_median  
0                4.3  
1                6.7  
2                7.2  
3                6.8  
4                6.9  


In [21]:
df4 = movies_data.groupby(['类型']).豆瓣网评分.agg( {'movie_type_rating_avg':np.mean ,
                                           'movie_type_max':np.max ,
                                           'movie_type_min':np.min ,
                                           'movie_type_median':np.median }).reset_index()
print(df4.head())

   类型  movie_type_rating_avg  movie_type_max  movie_type_min  \
0  传记               7.638390             9.7             2.2   
1  冒险               6.912956             9.4             2.1   
2  剧情               7.024251             9.8             2.1   
3  动作               6.579473             9.4             2.1   
4  历史               7.440551             9.6             2.4   

   movie_type_median  
0                7.7  
1                7.1  
2                7.2  
3                6.8  
4                7.6  


In [22]:
df5 = movies_data.groupby(['特色']).豆瓣网评分.agg( {'movie_type_rating_avg':np.mean ,
                                           'movie_type_max':np.max ,
                                           'movie_type_min':np.min ,
                                           'movie_type_median':np.median }).reset_index()
print(df5.head())

   特色  movie_type_rating_avg  movie_type_max  movie_type_min  \
0  励志               7.115536             9.6             2.4   
1  女性               7.030770             9.6             2.1   
2  感人               7.451520             9.7             2.2   
3  搞笑               6.286195             9.8             2.1   
4  文艺               7.082422             9.5             2.2   

   movie_type_median  
0                7.3  
1                7.1  
2                7.6  
3                6.5  
4                7.2  


In [23]:
df6 = movies_data.groupby(['地区']).豆瓣网评分.agg( {'movie_type_rating_avg':np.mean ,
                                           'movie_type_max':np.max ,
                                           'movie_type_min':np.min ,
                                           'movie_type_median':np.median }).reset_index()
print(df6.head())

     地区  movie_type_rating_avg  movie_type_max  movie_type_min  \
0  中国大陆               5.931474             9.4             2.1   
1    丹麦               7.218526             9.1             4.3   
2    伊朗               7.698985             9.2             6.1   
3   俄罗斯               7.368190             9.6             3.6   
4   加拿大               6.763789             9.6             2.6   

   movie_type_median  
0                6.3  
1                7.2  
2                7.8  
3                7.5  
4                6.8  


In [24]:
movies_data.head(2)

,类型,主演,地区,导演,特色,豆瓣网评分,电影名
0,剧情,徐峥|王传君|周一围|谭卓|章宇,中国大陆,文牧野,经典,8.9,我不是药神
1,剧情,冯小刚|许晴|张涵予|刘桦|李易峰,中国大陆,管虎,经典,7.8,老炮儿


### 对type, actors, director, trait四个类型数据分别构建一个频度统计字典，用于之后的one-hot编码

In [25]:
#意思就是对sequence连续使用function, 如果不给出initial, 
# 则第一次调用传递sequence的两个元素, 
# 以后把前一次调用的结果和sequence的下一个元素传递给function.
# 如果给出initial, 则第一次传递initial和sequence的第一个元素给function.
#reduce(lambda x,y: x+y, [1, 2, 3], 7) 
#输出 13 

In [26]:
# 首先形成key cout 对
# 对相同的key 的count进行累加
# list map is
#  [[('剧情', 1)], [('类型', 1)], [('剧情', 1)], [('剧情', 1)], [('剧情', 1)]]
# type_zip is    
# [('类型', 1), ('剧情', 4)]
# type_dict
#  {'类型': 1, '剧情': 4}
def reduce_func(x, y):
    for i in x:
        if i[0] == y[0][0]:
            x.remove(i)
            x.append(((i[0],i[1] + 1)))
            return x
    x.append(y[0])
    return x

lista = [['剧情','类型'], ['剧情'], ['剧情'], ['剧情']]
type_lista = [x for l in lista for x in l]
#['剧情', '类型', '剧情', '剧情', '剧情']
l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_lista))
type_zip = reduce(reduce_func, list(l))
type_dict = {}
for i in type_zip:
    type_dict[i[0]] = i[1]

NameError: name 'reduce' is not defined

In [27]:
from functools import reduce
def get_dim_dict(df, dim_name):
    # df[dim_name]) <class 'pandas.core.series.Series'>
    type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
    print('step 1\n',type_list[:10])
    type_list = [x for l in type_list for x in l]
    print('step 2\n',type_list[:10])
    def reduce_func(x, y):
        for i in x:
            if i[0] == y[0][0]:
                x.remove(i)
                x.append(((i[0],i[1] + 1)))
                return x
        x.append(y[0])
        return x
    l = filter(lambda x:x != None, map(lambda x:[(x, 1)], type_list))
    type_zip = reduce(reduce_func, list(l))
    #type_list = sorted(list(set(type_list)))
    #type_zip = zip(list(range(len(type_list))), type_list)
    type_dict = {}
    for i in type_zip:
        type_dict[i[0]] = i[1]
    return type_dict

In [28]:
type_dict = get_dim_dict(movies_data, '类型')

print(type_dict)

step 1
 [['剧情'], ['剧情'], ['剧情'], ['剧情'], ['剧情'], ['剧情'], ['剧情'], ['剧情'], ['剧情'], ['剧情']]
step 2
 ['剧情', '剧情', '剧情', '剧情', '剧情', '剧情', '剧情', '剧情', '剧情', '剧情']
{'剧情': 24275, '喜剧': 10960, '动作': 5271, '爱情': 15160, '科幻': 2772, '悬疑': 3237, '惊悚': 3984, '恐怖': 2036, '犯罪': 5051, '冒险': 3041, '音乐': 2426, '歌舞': 1141, '传记': 2037, '历史': 1778, '西部': 308, '战争': 1653, '武侠': 361, '灾难': 292, '奇幻': 3741}


In [29]:
#actors_dict = get_dim_dict(movies_data, '主演')
#print(actors_dict)



In [30]:
director_dict = get_dim_dict(movies_data, '导演')
print(director_dict)

step 1
 [['文牧野'], ['管虎'], ['陈思诚'], ['周申', '刘露'], ['叶伟民'], ['宁浩'], ['姜文'], ['李芳芳'], ['姜文'], ['肖央']]
step 2
 ['文牧野', '管虎', '陈思诚', '周申', '刘露', '叶伟民', '宁浩', '姜文', '李芳芳', '姜文']
{'克里斯托弗': 1, '麦奎里': 1, '吴天忍': 1, '高正': 1, '华永庄': 1, '罗真': 1, '徐谷明': 1, '胡兆洪': 1, '周力': 1, '许雷': 1, '保尔夫': 1, '张善琨': 1, '袁先': 1, '张晓颖': 1, '吴荫循': 1, 'Yingxun': 1, 'Wu': 1, '雷献禾': 1, '杨静': 1, '罗立言': 1, '林洪桐': 1, '张德托夫': 2, '王涛': 1, '齐兴家': 1, '艾水水': 1, '张忠华': 1, 'RAYIST': 1, '宋丽颖': 1, '杨弋枢': 1, '姬诚': 1, '杨惠龙': 1, '金丽妮': 1, '石建都': 1, '蒋骅': 1, '大知闲': 1, '胡力夫': 1, 'Feng': 1, 'Xiao': 1, '李佳伦': 1, '张涧赤': 1, '孙文学': 1, '王海丰': 1, '龙中华': 1, '杨潇': 1, '安占军': 1, '王加贺': 1, '石晓华': 2, '金也': 1, '上官醉': 1, 'Haoming': 1, 'Li': 1, '石学海': 1, 'lan': 1, 'Smithee': 1, '郑航': 1, '朱佳梦': 1, '邢树民': 1, 'Michelle': 1, 'Chen': 1, '阿岗': 1, '雅尔基': 1, '王文杰': 2, '王育臣': 1, '黎途': 1, '依莉琦': 1, '来赫': 1, '温晓龙': 1, '维克多': 1, '艾里斯': 1, '沃纳': 1, '赫尔佐格': 1, '吉姆': 1, '贾木许': 1, '阿基': 1, '考里斯马基': 1, '斯派克': 1, '维姆': 1, '文德斯': 1, '张大磊': 2, '周子阳': 1, '鹏飞': 1, '邢健': 1, '

In [31]:
trait_dict = get_dim_dict(movies_data, '特色')
print(trait_dict)

step 1
 [['经典'], ['经典'], ['经典'], ['经典'], ['经典'], ['经典'], ['经典'], ['经典'], ['经典'], ['经典']]
step 2
 ['经典', '经典', '经典', '经典', '经典', '经典', '经典', '经典', '经典', '经典']
{'黑帮': 3995, '青春': 16443, '感人': 3979, '经典': 17402, '文艺': 13824, '搞笑': 14089, '励志': 6250, '魔幻': 6818, '女性': 6724}
